# Explorando datos mundiales de COVID-19

In [ ]:
import numpy as np
import pandas as pd

## Preparación de datos

### Datos de casos confirmados

Primero, importamos los datos de casos confirmados a un dataframe

In [ ]:
df_confirmados = pd.read_csv('./data/covid19_confirmados.csv')
df_confirmados.head(25)



Las primeras cuatro columnas identifican el lugar y las 400 columnas siguientes corresponden al número de casos en ese lugar, durante 400 días consecutivos.

Algunos países tienen solo una fila, otros tienen varias, cada una correspondiente a una provincia o estado.

Luego, reduciremos la columna 'Province/State' y dejaremos una fila por país. Para facilitar esa reducción, crearemos un MultiIndex con las columnas 'Country/Region' y 'Province/State'.

In [ ]:
df_confirmados.set_index(keys=['Country/Region', 'Province/State'], drop=True,inplace=True)

In [ ]:
df_confirmados.head(5)

Pero antes de hacer esa reducción, ordenaremos un poco más el dataframe.

Eliminamos las columnas "Lat" y "Long":

In [ ]:
df_confirmados.drop(labels=['Lat', 'Long'], axis=1, inplace=True)

In [ ]:
df_confirmados.head(5)

Luego, reinterpretamos las etiquetas de fecha como Timestamps en lugar de simples objetos.

Para eso, le aplicamos a cada columna la función `pd.to_datetime`.

In [ ]:
df_confirmados.rename(mapper=pd.to_datetime, axis='columns', inplace=True);
df_confirmados.head(5)

Ahora sí reducimos el índice Province/State, para que quede solo una fila por país. Los valores para cada día serán la suma de casos dentro de las provincias para aquellos países en que habían varias provincias. Para los países que solo tenían una fila, el valor se mantiene. **TO DO**: Revisar que estas operaciones hayan quedado bien calculadas para ambos casos.

In [ ]:
df_confirmados = df_confirmados.groupby(by='Country/Region').aggregate(np.sum)

In [ ]:
df_confirmados.head(15)

### Datos de población por país

Importamos los datos, usando el país como índice principal.

In [ ]:
df_poblacion = pd.read_csv('./data/population_by_country_2020.csv', index_col='Country (or dependency)')
df_poblacion.head(5)

Ordenamos los países por orden alfabético.

In [ ]:
df_poblacion.sort_index(inplace=True)
df_poblacion.head(5)

Debemos encontrar las diferencias entre las etiquetas de los índices de los dataframes.

Primero, veamos los tamaños de ambos índices.

In [ ]:
print('n° filas df confirmados:', len(df_confirmados.index))
print('n° filas df población:', len(df_poblacion.index))
print('diferencia:', abs(len(df_confirmados.index) - len(df_poblacion.index)))

Ahora, veamos qué etiquetas están en el dataframe de confirmados pero no en el de población.

In [ ]:
df_confirmados.index.difference(df_poblacion.index)

In [ ]:
len(_)

Ahora lo contrario, es decir, cuáles están en población pero no en confirmados.

In [ ]:
df_poblacion.index.difference(df_confirmados.index)

In [ ]:
len(_)